## Multiclass Classification - Iris data

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.functional as F
import torch.optim as optim
import numpy as np
import random

random.seed(777)
torch.manual_seed(777)

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## Load Data & Preprocessing

In [3]:
iris = load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [4]:
train_X, test_X, train_y, test_y = train_test_split(iris.data, iris.target, test_size=0.3)

In [5]:
train_y

array([1, 0, 1, 2, 2, 2, 2, 2, 0, 1, 2, 2, 0, 0, 1, 2, 1, 1, 1, 2, 1, 0,
       1, 1, 2, 0, 1, 1, 0, 0, 1, 0, 2, 0, 1, 1, 2, 2, 2, 2, 2, 0, 0, 0,
       1, 2, 2, 2, 0, 1, 2, 2, 1, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 0, 0, 1, 2, 1, 0,
       1, 0, 1, 1, 1, 2, 2, 2, 0, 2, 2, 1, 2, 1, 1, 2, 2])

In [6]:
class makeData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [7]:
train_data = makeData(np.array(normalize(train_X)), np.array(train_y))
test_data = makeData(np.array(normalize(test_X)), np.array(test_y))

In [8]:
train_data.__getitem__(0)

(array([0.75384916, 0.31524601, 0.54825394, 0.17818253]), 1)

In [9]:
train_data.__len__()

105

## Modeling

In [10]:
class Multiclass_Classification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(Multiclass_Classification, self).__init__()
        
        self.FC1 = nn.Sequential(
                                nn.Linear(num_feature, 1000),
                                nn.ReLU()
                            )
        self.FC2 = nn.Sequential(
                                nn.Linear(1000, 500),
                                nn.ReLU()
                            )
        
        self.FC3 = nn.Sequential(
                                nn.Linear(500, 300),
                                nn.ReLU()
                            )
        self.dropout = nn.Dropout(0.5)
        
        self.out = nn.Linear(300, num_class)
        
    def forward(self, inputs):
        x = self.FC1(inputs)
        x = self.FC2(x)
        x = self.FC3(x)
        x = self.dropout(x)
        x = self.out(x)
        return x
    
    def predict(self, test_inputs):
        x = self.FC1(inputs)
        x = self.FC2(x)
        x = self.FC3(x)
        x = self.out(x)
        return torch.max(x, 1)[1]

## Training

In [11]:
EPOCHS = 2000
BATCH_SIZE = 20
NUM_FEATURE = 4
NUM_CLASS = 3

model = Multiclass_Classification(NUM_FEATURE, NUM_CLASS)
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS):
    for X_batch, y_batch in train_loader:
        inputs = Variable(torch.Tensor(X_batch.float()))
        targets = Variable(torch.LongTensor(y_batch.long()))
        model.zero_grad()
        y_pred = model(inputs)
        loss = criterion(y_pred, targets)
        loss.backward()
        optimizer.step()
    
    if epoch % 200 == 0:
        print(loss)

tensor(1.0654)
tensor(1.00000e-03 *
       9.1522)
tensor(1.00000e-02 *
       5.3697)
tensor(0.4586)
tensor(1.00000e-05 *
       3.5240)
tensor(1.00000e-03 *
       1.4986)
tensor(1.00000e-03 *
       4.7294)
tensor(1.00000e-03 *
       3.3923)
tensor(1.00000e-05 *
       1.9908)
tensor(1.00000e-02 *
       3.1028)


## Test Accuracy

In [12]:
test_loader = DataLoader(dataset=test_data, batch_size=1)
test_y_true = []
test_y_pred = []

for X_batch, y_batch in test_loader:
    inputs = Variable(torch.Tensor(X_batch.float()))
    test_y_true.append(y_batch.item())
    y_pred = model.predict(inputs)
    test_y_pred.append(int(y_pred.item()))

In [13]:
print(test_y_pred)

[0, 1, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 1, 2, 1, 2, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 2, 1, 0, 0, 0, 1, 0, 0, 0]


In [14]:
print(test_y_true)

[0, 1, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 1, 0, 2, 0, 1, 2, 0, 0, 1, 2, 1, 1, 0, 0, 2, 1, 0, 2, 2, 0, 1, 1, 2, 2, 1, 0, 0, 0, 1, 0, 0, 0]


In [15]:
accuracy_score(test_y_pred, test_y_true)

0.9777777777777777